In [70]:
from bs4 import BeautifulSoup as bs
import requests
url = 'http://sofifa.com/players?offset=0'

In [71]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

In [72]:
x = soup_maker(url)

In [99]:
def find_top_players(soup):
    table = soup.find('table', {'class': 'table table-hover persist-area'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a', {'class': ''})
    #print(all_a)
    #for player in all_a:
        #if player['href'][0:7] == '/player/':
    #    print(player['href'][0:8])
    #print(player['href'])
    #print('Hello')
    return(['http://sofifa.com' + player['href'] for player in all_a if player['href'][0:8] == '/player/'])

In [100]:
h = find_top_players(x)

In [101]:
type(h)

list

In [102]:
h

['http://sofifa.com/player/158023',
 'http://sofifa.com/player/20801',
 'http://sofifa.com/player/190871',
 'http://sofifa.com/player/176580',
 'http://sofifa.com/player/167495',
 'http://sofifa.com/player/193080',
 'http://sofifa.com/player/192985',
 'http://sofifa.com/player/188545',
 'http://sofifa.com/player/183277',
 'http://sofifa.com/player/182521',
 'http://sofifa.com/player/178603',
 'http://sofifa.com/player/167664',
 'http://sofifa.com/player/155862',
 'http://sofifa.com/player/211110',
 'http://sofifa.com/player/200389',
 'http://sofifa.com/player/192119',
 'http://sofifa.com/player/177003',
 'http://sofifa.com/player/168542',
 'http://sofifa.com/player/153079',
 'http://sofifa.com/player/138956',
 'http://sofifa.com/player/1179',
 'http://sofifa.com/player/215914',
 'http://sofifa.com/player/202126',
 'http://sofifa.com/player/195864',
 'http://sofifa.com/player/194765',
 'http://sofifa.com/player/190460',
 'http://sofifa.com/player/189509',
 'http://sofifa.com/player/1849

In [164]:
from bs4 import BeautifulSoup as bs
import requests
import re

url = 'http://sofifa.com/players?offset=0'


fifa_stats = ['Crossing', 'Finishing', 'Heading Accuracy',
              'Short Passing', 'Volleys', 'Dribbling', 'Curve',
              'Free Kick Accuracy', 'Long Passing', 'Ball Control',
              'Acceleration', 'Sprint Speed', 'Agility', 'Reactions',
              'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength',
              'Long Shots', 'Aggression', 'Interceptions', 'Positioning',
              'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle',
              'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking',
              'GK Positioning', 'GK Reflexes']


def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup


def find_top_players(soup):
    final_details = {}
    table = soup.find('table', {'class': 'table table-hover persist-area'})
    tbody = table.find('tbody')
    all_a = tbody.find_all('a')
    #print(all_a)
    for player in all_a:
        #print(player['href'])
        final_details['short_name'] = player.text
        #print(final_details)
        if player['href'][0:8] == '/player/':
            final_details.update(player_all_details('http://sofifa.com' + player['href']))
        #print(final_details)


def find_player_info(soup):
    player_data = {}
    player_data['image'] = soup.find('img')['data-src']
    player_data['full_name'] = soup.find('h1').text.split(' (')[0]
    span = soup.find('span', attrs={'class': None}).text.strip()
    #span = soup.find('span').text.strip()
    #print(soup)
    #print(span)
    dob = re.search('(\(.*)\)', span).group(0)
    player_data['dob'] = dob.replace('(', '').replace(')', '')
    #print(span)
    infos = span.replace(dob + ' ', '').split(' ')
    #infos = span.split('Age')
    infos.append(infos[-1])
    infos[-2] = infos[-3]
    infos[-3] = infos[-4]
    infos[-4] = infos[-5][-3:-1]
    infos[-4] = infos[-4]+infos[-5][-1]
    infos[-5] = infos[-5][:-3]
    #print(infos)
    player_data['pref_pos'] = infos[:infos.index('Age')]
    #print(player_data['pref_pos'])
    player_data['age'] = int(infos[infos.index('Age') + 1: -2][0])
    player_data['height'] = int((infos[infos.index('Age') + 2: -1][0]).replace('cm', ''))
    player_data['weight'] = int((infos[infos.index('Age') + 3:][0]).replace('kg', ''))
    return(player_data)


#def find_player_stats(soup):
#    player_data = {}
#    info = re.findall('\d+', soup.text)
#    player_data['rating'] = int(info[0])
#    player_data['potential'] = int(info[1])
#    player_data['value'] = int(info[2])
#    player_data['wage'] = int(info[3])
#    return(player_data)

def find_player_stats(soup):
    player_data = {}
    info = re.findall('\d+', soup.text)
    player_data['rating'] = int(info[0])
    player_data['potential'] = int(info[1])
    player_data['value'] = int(info[2])
    player_data['wage'] = int(info[4])*1000
    return(player_data)

def find_player_secondary_info(soup):
    player_data = {}
    player_data['preff_foot'] = soup.find('label', text='Preferred Foot')\
        .parent.contents[2].strip('\n ')
    player_data['club'] = soup.find_all('ul')[1].find('a').text
    player_data['club_pos'] = soup.find('label', text='Position')\
        .parent.find('span').text
    player_data['club_jersey'] = soup.find('label', text='Jersey number')\
        .parent.contents[2].strip('\n ')
    if soup.find('label', text='Joined'):
        player_data['club_joined'] = soup.find('label', text='Joined')\
            .parent.contents[2].strip('\n ')
    player_data['contract_valid'] = soup.find(
        'label', text='Contract valid until')\
        .parent.contents[2].strip('\n ')
    if len(soup.find_all('ul')) > 2:
        player_data['country'] = soup.find_all('ul')[2].find('a').text
    return(player_data)


def find_fifa_info(soup):
    player_data = {}
    divs_without_skill = soup[1].find_all('div', {'class': 'col-3'})[:3]
    more_lis = [div.find_all('li') for div in divs_without_skill]
    lis = soup[0].find_all('li') + more_lis[0]
    for li in lis:
        for stats in fifa_stats:
            if stats in li.text:
                player_data[stats.replace(' ', '_').lower()] = int(
                    (li.text.split(' ')[0]).replace('\n', ''))
    traits = soup[1].find('h4', text='Traits')
    if traits:
        player_data['traits'] = [li.text.replace('\xa0', '') for li in
            traits.parent.next_sibling.next_sibling.find_all('li')]
    specialities = soup[1].find('h4', text='Specialities')
    if specialities:
        player_data['specialities'] = [li.text.replace('\xa0', '') for li in
            specialities.parent.next_sibling.next_sibling.find_all('li')]
    return(player_data)


def player_all_details(url):
    all_details = {}
    #print(url)
    soup = soup_maker(url)
    player_info = soup.find('div', {'class': 'player'})
    #print(player_info)
    all_details.update(find_player_info(player_info))
    player_stats = soup.find('div', {'class': 'stats'})
    all_details.update(find_player_stats(player_stats))
    secondary_info = soup.find('div', {'class': 'teams'})
    all_details.update(find_player_secondary_info(secondary_info))
    fifa_info = soup.find_all('div', {'class': 'columns mb-20'})
    all_details.update(find_fifa_info(fifa_info))
    return(all_details)

In [165]:
soup = soup_maker(url)
find_top_players(soup)

['Lionel', 'Messi', '', 'CF', 'ST', 'RW', 'Age', '30', '170cm', '72kg']


AttributeError: 'NoneType' object has no attribute 'parent'